In [6]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [7]:
adeft_grounder = AdeftGrounder()

In [8]:
shortforms = ['DLK']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [21]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 0]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [22]:
list(zip(longforms, counts))

[('dual leucine zipper kinase', 67),
 ('dual leucine zipper bearing kinase', 24),
 ('diffuse lamellar keratitis', 18),
 ('dual leucine kinase', 4),
 ('drosophila leucokinin', 1),
 ('dual lencine zipper kinase', 1),
 ('duel leucine zipper kinase', 1),
 ('duel leucine zipper bearing kinase', 1),
 ('dual zipper bearing kinase', 1)]

In [13]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [14]:
names

{'MESH:D007634': 'Keratitis',
 'HGNC:6851': 'MAP3K12',
 'HGNC:2676': 'DAPK3',
 'HGNC:2907': 'DLK1'}

In [25]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

In [26]:
result = [grounding_map, names, pos_labels]

In [27]:
result

[{'diffuse lamellar keratitis': 'ungrounded',
  'drosophila leucokinin': 'ungrounded',
  'dual lencine zipper kinase': 'HGNC:6851',
  'dual leucine kinase': 'HGNC:6851',
  'dual leucine zipper bearing kinase': 'HGNC:6851',
  'dual leucine zipper kinase': 'HGNC:6851',
  'dual zipper bearing kinase': 'HGNC:6851',
  'duel leucine zipper bearing kinase': 'HGNC:6851',
  'duel leucine zipper kinase': 'HGNC:6851'},
 {'HGNC:6851': 'MAP3K12'},
 ['HGNC:6851']]

In [28]:
grounding_map, names, pos_labels = [{'diffuse lamellar keratitis': 'ungrounded',
  'drosophila leucokinin': 'ungrounded',
  'dual lencine zipper kinase': 'HGNC:6851',
  'dual leucine kinase': 'HGNC:6851',
  'dual leucine zipper bearing kinase': 'HGNC:6851',
  'dual leucine zipper kinase': 'HGNC:6851',
  'dual zipper bearing kinase': 'HGNC:6851',
  'duel leucine zipper bearing kinase': 'HGNC:6851',
  'duel leucine zipper kinase': 'HGNC:6851'},
 {'HGNC:6851': 'MAP3K12'},
 ['HGNC:6851']]

In [29]:
excluded_longforms = []

In [30]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [31]:
additional_entities = {'HGNC:2907': ['DLK1', ['DLK', 'DLK1', 'delta-like homolog']]}

In [32]:
unambiguous_agent_texts = {}

In [33]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [34]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [35]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [36]:
intersection1

[('HGNC:2907', 'HGNC:2907', 151)]

In [37]:
intersection2

[('HGNC:6851', 'HGNC:2907', 0), ('ungrounded', 'HGNC:2907', 0)]

In [38]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [39]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [40]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-11-10 04:45:04] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-11-10 04:45:07] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9663521772345302 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [41]:
classifier.stats

{'label_distribution': {'HGNC:6851': 79, 'ungrounded': 13, 'HGNC:2907': 85},
 'f1': {'mean': 0.966352, 'std': 0.024766},
 'precision': {'mean': 0.966768, 'std': 0.020269},
 'recall': {'mean': 0.969318, 'std': 0.027535},
 'HGNC:6851': {'f1': {'mean': 0.969675, 'std': 0.027154},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 0.942484, 'std': 0.051198}},
 'HGNC:2907': {'f1': {'mean': 0.963225, 'std': 0.023601},
  'pr': {'mean': 0.941176, 'std': 0.052613},
  'rc': {'mean': 0.988889, 'std': 0.022222}},
 'ungrounded': {'f1': {'mean': 0.933333, 'std': 0.133333},
  'pr': {'mean': 0.9, 'std': 0.2},
  'rc': {'mean': 1.0, 'std': 0.0}}}

In [42]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [43]:
disamb.dump(model_name, results_path)

In [44]:
print(disamb.info())

Disambiguation model for DLK

Produces the disambiguations:
	DLK1*	HGNC:2907
	MAP3K12*	HGNC:6851

Class level metrics:
--------------------
Grounding	Count	F1     
   DLK1*	85	0.96322
MAP3K12*	79	0.96968
Ungrounded	13	0.93333

Weighted Metrics:
-----------------
	F1 score:	0.96635
	Precision:	0.96677
	Recall:		0.96932

* Positive labels
See Docstring for explanation



In [50]:
model_to_s3(disamb)

In [30]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [31]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:10967']

In [36]:
texts[3]

'The non-neuronal monoamine transporters (OCT1, OCT2, EMT, and PMAT) play a key role in the clearance of monoamines from extracellular compartments. In a previous report we described endometrial distribution and cyclic variation of the vesicular monoamine transporter (VMAT2) mRNA and the neuronal norepinephrine transporter (NET) mRNA. In the present study we used in situ hybridization, real-time PCR and immunohistochemistry to reveal tissue distribution and cyclic variation of mRNA for the non-neuronal monoamine transporters in the human endometrium and early pregnancy decidua. We found that non-neuronal monoamine transporters are predominantly expressed in the stroma. The plasma membrane monoamine transporter (PMAT) mRNA expression peaked in the proliferative phase, whereas the extra-neuronal monoamine transporter (EMT) mRNA expression peaked in the secretory phase. The organic cation transporter 2 (OCT2) mRNA expression was exclusively detected in few scattered stromal cells and OCT1

In [46]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:2907']

In [47]:
len(texts)

7

In [48]:
'\n\n'.join(texts)

"To investigate the effect of the tissue origin of stromal fibroblasts and epithelial cells on reconstructed corneas in vitro. Four types of constructs were produced by the self-assembly approach using the following combinations of human cells: corneal fibroblasts/corneal epithelial cells, corneal fibroblasts/skin epithelial cells, skin fibroblasts/corneal epithelial cells, skin fibroblasts/skin epithelial cells. Fibroblasts were cultured with ascorbic acid to produce stromal sheets on which epithelial cells were cultured. After 2 weeks at the air-liquid interface, the reconstructed tissues were photographed, absorption spectra were measured, and tissues were fixed for histologic analysis. Cytokine expression in corneal- or skin-fibroblast-conditioned media was determined with the use of protein array membranes. The effect of culturing reconstructed tissues with conditioned media, or media supplemented with a cytokine secreted mainly by corneal fibroblasts, was determined. The tissue s

In [49]:
texts[0]

'To investigate the effect of the tissue origin of stromal fibroblasts and epithelial cells on reconstructed corneas in vitro. Four types of constructs were produced by the self-assembly approach using the following combinations of human cells: corneal fibroblasts/corneal epithelial cells, corneal fibroblasts/skin epithelial cells, skin fibroblasts/corneal epithelial cells, skin fibroblasts/skin epithelial cells. Fibroblasts were cultured with ascorbic acid to produce stromal sheets on which epithelial cells were cultured. After 2 weeks at the air-liquid interface, the reconstructed tissues were photographed, absorption spectra were measured, and tissues were fixed for histologic analysis. Cytokine expression in corneal- or skin-fibroblast-conditioned media was determined with the use of protein array membranes. The effect of culturing reconstructed tissues with conditioned media, or media supplemented with a cytokine secreted mainly by corneal fibroblasts, was determined. The tissue s